In [ ]:
import csv
import os
import requests

# Define the API endpoint and parameters
API_URL = "https://api.openchargemap.io/v3/poi/"
API_KEY = "36e92dd8-2f87-4ba2-909c-58f81f7ed881"
params = {
    "output": "json",
    "countrycode": "AU",
    "maxresults": 500,  # Increase max results to ensure all stations are retrieved
    "key": API_KEY  # Your API key
}

# Function to fetch EV station data from the API
def fetch_ev_stations(api_url, params):
    try:
        response = requests.get(api_url, params=params)
        response.raise_for_status()  # Raises HTTPError for bad responses
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch data: {e}")
        return None

# Function to save EV station data to a CSV file
def save_to_csv(data, csv_file_path):
    try:
        with open(csv_file_path, mode="w", newline="", encoding="utf-8") as csv_file:
            csv_writer = csv.writer(csv_file)
            # Write header row
            csv_writer.writerow([
                "ID", "UsageTypeID", "UsageCost", "NumberOfPoints", "StatusTypeID",
                "PhonePrimaryContact", "ContactEmail", "IsPayAtLocation", "UsageTypeTitle",
                "IsOperational", "AddressTitle", "AddressLine1", "AddressLine2", "Town",
                "StateOrProvince", "Postcode", "Latitude", "Longitude",
                "ConnectionID", "ConnectionTypeTitle", "LevelTitle", "PowerKW",
                "CurrentTypeTitle", "Quantity"
            ])

            # Process and write each station's data to CSV
            for station in data:
                station_id = station.get("ID", "N/A")
                usage_type_id = station.get("UsageTypeID", "N/A")
                
                # Check if "UsageCost" exists and is not None
                usage_cost = station.get("UsageCost", "Cost information not available")
                
                number_of_points = station.get("NumberOfPoints", "N/A")
                status_type_id = station.get("StatusTypeID", "N/A")
                
                operator_info = station.get("OperatorInfo", {})
                phone_primary_contact = operator_info.get("PhonePrimaryContact", "N/A")
                contact_email = operator_info.get("ContactEmail", "N/A")
                
                usage_type = station.get("UsageType", {})
                is_pay_at_location = usage_type.get("IsPayAtLocation", "N/A")
                usage_type_title = usage_type.get("Title", "N/A")
                
                status_type = station.get("StatusType", {})
                is_operational = status_type.get("IsOperational", "N/A")
                
                address_info = station.get("AddressInfo", {})
                address_title = address_info.get("Title", "N/A")
                address_line1 = address_info.get("AddressLine1", "N/A")
                address_line2 = address_info.get("AddressLine2", "N/A")
                town = address_info.get("Town", "N/A")
                state_or_province = address_info.get("StateOrProvince", "N/A")
                postcode = address_info.get("Postcode", "N/A")
                latitude = address_info.get("Latitude", "N/A")
                longitude = address_info.get("Longitude", "N/A")

                # Iterate over each connection
                connections = station.get("Connections", [])
                for connection in connections:
                    connection_id = connection.get("ID", "N/A")
                    connection_type = connection.get("ConnectionType", {})
                    connection_type_title = connection_type.get("Title", "N/A")
                    
                    level = connection.get("Level", {})
                    level_title = level.get("Title", "N/A")
                    
                    power_kw = connection.get("PowerKW", "N/A")
                    
                    current_type = connection.get("CurrentType", {})
                    current_type_title = current_type.get("Title", "N/A")
                    
                    quantity = connection.get("Quantity", "N/A")
                    
                    # Write each connection as a new row
                    csv_writer.writerow([
                        station_id, usage_type_id, usage_cost, number_of_points, status_type_id,
                        phone_primary_contact, contact_email, is_pay_at_location, usage_type_title,
                        is_operational, address_title, address_line1, address_line2, town,
                        state_or_province, postcode, latitude, longitude,
                        connection_id, connection_type_title, level_title, power_kw,
                        current_type_title, quantity
                    ])

        print(f"CSV file saved successfully: {csv_file_path}")
    except Exception as e:
        print(f"Error occurred while creating the CSV file: {e}")

# Fetch and save EV station data
data = fetch_ev_stations(API_URL, params)
if data:
    desktop_path = os.path.join(os.path.expanduser("~"), "Desktop", "capstoneB")
    os.makedirs(desktop_path, exist_ok=True)  # Create the folder if it doesn't exist
    csv_file_path = os.path.join(desktop_path, "ev_stations.csv")
    save_to_csv(data, csv_file_path)
else:
    print("No data to save.")
